In [1]:
!pip install transformers[torch,sentencepiece] langchain langchain-community langchain-huggingface faiss-cpu jq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 737.4/737.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.5 MB/s eta 0:00:00


In [2]:
from google.colab import userdata

HUGGINGFACE_TOKEN = userdata.get('HF_TOKEN_READ')
!huggingface-cli login --token $HUGGINGFACE_TOKEN

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
The token `LLM_Course` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `LLM_Course`


In [3]:
from transformers.trainer_utils import set_seed


set_seed(42)

### LangChainでLLMを使う

In [4]:
import torch
from langchain_huggingface import HuggingFacePipeline
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    pipeline
)

In [5]:
# Hugging Face Hubにおけるモデル名を指定
model_name = "llm-book/Swallow-7b-hf-oasst1-21k-ja"

# モデルを読み込む
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

# テキスト生成用のパラメータを指定
generation_config = {
    "max_new_tokens": 128,
    "do_sample": False,
    "temperature": None,
    "top_p": None
}

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/761 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/914k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.30M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

In [6]:
text_generation_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    **generation_config
)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

Device set to use cuda:0


In [7]:
tokenizer.chat_template

"{%- for message in messages %}\n{%- if message['role'] == 'user' %}\n{{ bos_token + 'ユーザ：' + message['content'] + eos_token }}\n{%- elif message['role'] == 'assistant' %}\n{{ bos_token + 'アシスタント：'  + message['content'] + eos_token }}\n{%- endif %}\n{% if loop.last and add_generation_prompt %}\n{{ bos_token + 'アシスタント：' }}\n{%- endif %}\n{% endfor %}"

In [8]:
from pprint import pprint

# モデルに入力する会話データ
llm_prompt_messages = [
    {"role": "user", "content": "四国地方で一番高い山は？"},
]

llm_prompt_text = tokenizer.apply_chat_template(
    llm_prompt_messages,
    tokenize=False,
    add_generation_prompt=True,
)
print(llm_prompt_text)

<s>ユーザ：四国地方で一番高い山は？</s><s>アシスタント：


In [9]:
llm_output_message = llm.invoke(llm_prompt_text)
print(llm_output_message)

<s>ユーザ：四国地方で一番高い山は？</s><s>アシスタント：四国地方で一番高い山は、徳島県と高知県の県境にある剣山で、標高は1,955メートルです。


In [10]:
llm.invoke("四国地方で一番高い山は？<s>アシスタント：")

'四国地方で一番高い山は？<s>アシスタント：四国地方で一番高い山は、徳島県と高知県の県境にある剣山で、標高は1,955メートルです。 '

### Chat Modelコンポーネントの利用

In [11]:
from langchain_huggingface import ChatHuggingFace

chat_model = ChatHuggingFace(llm=llm, tokenizer=tokenizer)

In [12]:
from langchain_core.messages import HumanMessage, SystemMessage

chat_message = [HumanMessage(content="四国地方で一番高い山は？")]

chat_prompt = chat_model._to_chat_prompt(chat_message)
print(chat_prompt)

<s>ユーザ：四国地方で一番高い山は？</s><s>アシスタント：


In [13]:
chat_output_message = chat_model.invoke(chat_message)
pprint(chat_output_message)

AIMessage(content='<s>ユーザ：四国地方で一番高い山は？</s><s>アシスタント：四国地方で一番高い山は、徳島県と高知県の県境にある剣山で、標高は1,955メートルです。', additional_kwargs={}, response_metadata={}, id='run-b11c1f76-5bfd-421b-ad59-043e5ed5878f-0')


In [14]:
response_text = chat_output_message.content[len(chat_prompt):]
print(response_text)

四国地方で一番高い山は、徳島県と高知県の県境にある剣山で、標高は1,955メートルです。


### Chainを構築する

In [15]:
from langchain_core.prompts import ChatPromptTemplate


# 任意のクエリからプロンプトを構築するPrompt Templateを作成
prompt_template = ChatPromptTemplate.from_messages(
    [("user", "{query}")]
)

prompt_template_output = prompt_template.invoke(
    {"query": "四国地方で一番高い山は？"}
)
pprint(prompt_template_output)

ChatPromptValue(messages=[HumanMessage(content='四国地方で一番高い山は？', additional_kwargs={}, response_metadata={})])


In [16]:
# Prompt TemplateとChat Modelを連結したChainを作成
chain = prompt_template | chat_model

# Chainを実行し、結果を確認
chain_output = chain.invoke({"query": "四国地方で一番高い山は？"})
pprint(chain_output)

AIMessage(content='<s>ユーザ：四国地方で一番高い山は？</s><s>アシスタント：四国地方で一番高い山は、徳島県と高知県の県境にある剣山で、標高は1,955メートルです。', additional_kwargs={}, response_metadata={}, id='run-cf67c7cf-0ae0-4747-b9a6-24468d51c50c-0')


In [18]:
from langchain_core.output_parsers import StrOutputParser

output_chain = prompt_template | chat_model | StrOutputParser()

output_chain.invoke({"query": "四国地方で一番高い山は？"})

'<s>ユーザ：四国地方で一番高い山は？</s><s>アシスタント：四国地方で一番高い山は、徳島県と高知県の県境にある剣山で、標高は1,955メートルです。'

In [22]:
from langchain_core.prompt_values import ChatPromptValue
from langchain_core.runnables import RunnableLambda

def chat_model_resp_only_func(chat_prompt_value: ChatPromptValue) -> str:
    """chat_modelにchat_prompt_valueを入力し、出力からモデルの応答部分のみを文字列で返す"""
    chat_prompt = chat_model._to_chat_prompt(
        chat_prompt_value.messages
    )
    chat_output_message = chat_model.invoke(chat_prompt_value)
    response_text = chat_output_message.content[len(chat_prompt):]
    return response_text

# 定義した関数の処理を行うRunnableを作成
chat_model_resp_only = RunnableLambda(chat_model_resp_only_func)

chain_resp_only = prompt_template | chat_model_resp_only

# Chainを実行し、結果を確認
chain_resp_only_output = chain_resp_only.invoke(
    {"query": "四国地方で一番高い山は？"}
)
print(chain_resp_only_output)

四国地方で一番高い山は、徳島県と高知県の県境にある剣山で、標高は1,955メートルです。


### LangChainで文埋め込みモデルを使う

In [24]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

# Huggingface hubにおけるモデルの指定
embedding_model_name = "BAAI/bge-m3"

embedding_model = HuggingFaceEmbeddings(
    model_name=embedding_model_name,
    model_kwargs={"model_kwargs": {"torch_dtype": torch.float16}},
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [25]:
sample_texts = [
    "日本で一番高い山はなんですか？",
    "日本で一番高い山は富士山です。"
]

sample_embeddings = embedding_model.embed_documents(sample_texts)
print(sample_embeddings)

[[0.00482940673828125, 0.03375244140625, -0.02288818359375, -0.028778076171875, 0.0111846923828125, -0.031219482421875, -0.01186370849609375, -0.0078277587890625, -0.03509521484375, -0.0262908935546875, -0.0206756591796875, 0.0282440185546875, -0.0189056396484375, 0.00675201416015625, 0.045074462890625, -0.026275634765625, 0.03656005859375, -0.0029201507568359375, -0.005413055419921875, 0.024261474609375, -0.0080108642578125, -0.0209503173828125, -0.027099609375, 0.0288848876953125, 0.009735107421875, 0.01264190673828125, -0.01316070556640625, 0.0038051605224609375, -0.006317138671875, -0.042938232421875, -0.023284912109375, -0.0234375, 0.005168914794921875, -0.0009760856628417969, -0.027984619140625, -0.01221466064453125, 0.0016584396362304688, -0.033111572265625, -0.03985595703125, 0.0008668899536132812, 0.03399658203125, 0.01806640625, 0.041656494140625, -0.026763916015625, -0.043121337890625, -0.0199127197265625, -0.0153656005859375, -0.032806396484375, 0.0181732177734375, -0.01262

In [26]:
similarity = torch.nn.functional.cosine_similarity(
    torch.tensor([sample_embeddings[0]]),
    torch.tensor([sample_embeddings[1]])
)

print(similarity)

tensor([0.7640])


### LangChainでRAGを実装する

In [27]:
# 検索対象の文書集合のファイルをダウンロード
!wget \
https://github.com/ghmagazine/llm-book/raw/main/chapter13/docs.json

--2025-01-12 21:51:54--  https://github.com/ghmagazine/llm-book/raw/main/chapter13/docs.json
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ghmagazine/llm-book/main/chapter13/docs.json [following]
--2025-01-12 21:51:55--  https://raw.githubusercontent.com/ghmagazine/llm-book/main/chapter13/docs.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1178382 (1.1M) [text/plain]
Saving to: ‘docs.json’

docs.json           100%[===================>]   1.12M  4.54MB/s    in 0.2s    

2025-01-12 21:51:56 (4.54 MB/s) - ‘docs.json’ saved [1178382/1178382]



In [31]:
from langchain_community.document_loaders import JSONLoader

document_loader = JSONLoader(
    file_path="./docs.json",
    jq_schema=".text",
    json_lines=True,
)

documents = document_loader.load()

In [32]:
len(documents)

103

In [33]:
pprint(documents[0])

Document(metadata={'source': '/content/docs.json', 'seq_num': 1}, page_content='富士山（ふじさん）は、静岡県（富士宮市、富士市、裾野市、御殿場市、駿東郡小山町）と山梨県（富士吉田市、南都留郡鳴沢村）に跨る活火山である。標高3776.12 m、日本最高峰（剣ヶ峰）の独立峰で、その優美な風貌は日本国外でも日本の象徴として広く知られている。 数多くの芸術作品の題材とされ芸術面のみならず、気候や地層など地質学的にも社会に大きな影響を与えている。懸垂曲線の山容を有した玄武岩質成層火山で構成され、その山体は駿河湾の海岸まで及ぶ。 古来より霊峰とされ、特に山頂部は浅間大神が鎮座するとされたため、神聖視された。噴火を沈静化するため律令国家により浅間神社が祭祀され、浅間信仰が確立された。また、富士山修験道の開祖とされる富士上人により修験道の霊場としても認識されるようになり、登拝が行われるようになった。これら富士信仰は時代により多様化し、村山修験や富士講といった一派を形成するに至る。現在、富士山麓周辺には観光名所が多くある他、夏季シーズンには富士登山が盛んである。 日本三名山（三霊山）、日本百名山、日本の地質百選に選定されている。また、1936年（昭和11年）には富士箱根伊豆国立公園に指定されている。その後、1952年（昭和27年）に特別名勝、2011年（平成23年）に史跡、さらに2013年（平成25年）6月22日には関連する文化財群とともに「富士山-信仰の対象と芸術の源泉」の名で世界文化遺産に登録された。 富士山についての最も古い記録は『常陸国風土記』における「福慈岳」という語であると言われている。他にも多くの呼称が存在し、不二山もしくは不尽山と表記する古文献もある。また、『竹取物語』における伝説もある。「フジ」という長い山の斜面を表す大和言葉から転じて富士山と称されたという説もある。近代以降の語源説としては、宣教師バチェラーは、名前は「火を噴く山」を意味するアイヌ語の「フンチヌプリ」に由来するとの説を提示した。しかし、これは囲炉裏の中に鎮座する火の姥神を表す「アペフチカムイ」からきた誤解であるとの反論がある。その他の語源説として、マレー語説、マオリ語説、原ポリネシア語説がある。 明確に「富士山」と表記さ

In [34]:
# 文字数の確認
print(len(documents[0].page_content))

21232


#### textの分割

In [36]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=100,
    add_start_index=True
)

split_documents = text_splitter.split_documents(documents)

print(len(split_documents))

1475


In [37]:
# 分割後の文書の内容を確認
pprint(split_documents[0])
pprint(split_documents[1])

Document(metadata={'source': '/content/docs.json', 'seq_num': 1, 'start_index': 0}, page_content='富士山（ふじさん）は、静岡県（富士宮市、富士市、裾野市、御殿場市、駿東郡小山町）と山梨県（富士吉田市、南都留郡鳴沢村）に跨る活火山である。標高3776.12 m、日本最高峰（剣ヶ峰）の独立峰で、その優美な風貌は日本国外でも日本の象徴として広く知られている。 数多くの芸術作品の題材とされ芸術面のみならず、気候や地層など地質学的にも社会に大きな影響を与えている。懸垂曲線の山容を有した玄武岩質成層火山で構成され、その山体は駿河湾の海岸まで及ぶ。')
Document(metadata={'source': '/content/docs.json', 'seq_num': 1, 'start_index': 129}, page_content='数多くの芸術作品の題材とされ芸術面のみならず、気候や地層など地質学的にも社会に大きな影響を与えている。懸垂曲線の山容を有した玄武岩質成層火山で構成され、その山体は駿河湾の海岸まで及ぶ。 古来より霊峰とされ、特に山頂部は浅間大神が鎮座するとされたため、神聖視された。噴火を沈静化するため律令国家により浅間神社が祭祀され、浅間信仰が確立された。また、富士山修験道の開祖とされる富士上人により修験道の霊場としても認識されるようになり、登拝が行われるようになった。これら富士信仰は時代により多様化し、村山修験や富士講といった一派を形成するに至る。現在、富士山麓周辺には観光名所が多くある他、夏季シーズンには富士登山が盛んである。')


In [39]:
# 分割後の文書の長さを確認
pprint(len(split_documents[0].page_content))
pprint(len(split_documents[1].page_content))

221
310


### ベクトルインデックスの作成

In [41]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(split_documents, embedding_model)

print(vectorstore.index.ntotal)

1475


### Retrieverコンポーネントの作成

In [43]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [44]:
retrieved_documents = retriever.invoke("四国地方で一番高い山は？")
pprint(retrieved_documents)

[Document(id='c3ebf1b1-a935-436e-a693-ee4f8543ec6f', metadata={'source': '/content/docs.json', 'seq_num': 26, 'start_index': 0}, page_content='この項目に含まれる文字「鎚」は、オペレーティングシステムやブラウザなどの環境により表示が異なります。 石鎚山（いしづちさん、いしづちやま）は、四国山地西部に位置する標高1,982 mの山で、近畿以西を「西日本」とした場合の西日本最高峰で、山頂から望む展望が四国八十八景64番に選定。愛媛県西条市と久万高原町の境界に位置する。 石鉄山、石鈇山、石土山、石槌山とも表記され、伊予の高嶺とも呼ばれる。『日本霊異記』には「石槌山」と記され、延喜式の神名帳（延喜式神名帳）では「石鉄神社」と記されている。前神寺および横峰寺では「石鈇山（しゃくまざん）」とも呼ぶ。'),
 Document(id='1e29f5ba-ae24-4fd3-a89f-1b236ea32bfc', metadata={'source': '/content/docs.json', 'seq_num': 1, 'start_index': 0}, page_content='富士山（ふじさん）は、静岡県（富士宮市、富士市、裾野市、御殿場市、駿東郡小山町）と山梨県（富士吉田市、南都留郡鳴沢村）に跨る活火山である。標高3776.12 m、日本最高峰（剣ヶ峰）の独立峰で、その優美な風貌は日本国外でも日本の象徴として広く知られている。 数多くの芸術作品の題材とされ芸術面のみならず、気候や地層など地質学的にも社会に大きな影響を与えている。懸垂曲線の山容を有した玄武岩質成層火山で構成され、その山体は駿河湾の海岸まで及ぶ。'),
 Document(id='de62dd66-856b-4189-bd80-0cf93579b782', metadata={'source': '/content/docs.json', 'seq_num': 96, 'start_index': 0}, page_content='四阿山（あずまやさん）は、長野県と群馬県の県境に跨る山。標高2,354 m。日本百名山の一つに数えられている。吾妻山・吾嬬山（あがつま

### RAGのChainの構築

In [46]:
rag_prompt_text = (
    "以下の文書の内容を参考にして、質問に答えてください。\n\n"
    "---n\{context}\n---\n\n質問: {query}"
)
rag_prompt_template = ChatPromptTemplate.from_messages(
    [("user", rag_prompt_text)]
)

In [48]:
from langchain_core.documents import Document
import logging

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)


def format_documents_func(documents: list[Document]) -> str:
    """文書のリストを改行で連結した1つの文字列として返す"""
    logger.info(documents)
    return "\n\n".join(
        document.page_content for document in documents
    )

format_documents = RunnableLambda(format_documents_func)

In [51]:
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {
        "context": retriever | format_documents,
        "query": RunnablePassthrough(),
    }
    | rag_prompt_template
    | chat_model_resp_only
)

In [53]:
rag_chain_output = rag_chain.invoke("四国地方で一番高い山は？")
print(rag_chain_output)

INFO:__main__:[Document(id='c3ebf1b1-a935-436e-a693-ee4f8543ec6f', metadata={'source': '/content/docs.json', 'seq_num': 26, 'start_index': 0}, page_content='この項目に含まれる文字「鎚」は、オペレーティングシステムやブラウザなどの環境により表示が異なります。 石鎚山（いしづちさん、いしづちやま）は、四国山地西部に位置する標高1,982 mの山で、近畿以西を「西日本」とした場合の西日本最高峰で、山頂から望む展望が四国八十八景64番に選定。愛媛県西条市と久万高原町の境界に位置する。 石鉄山、石鈇山、石土山、石槌山とも表記され、伊予の高嶺とも呼ばれる。『日本霊異記』には「石槌山」と記され、延喜式の神名帳（延喜式神名帳）では「石鉄神社」と記されている。前神寺および横峰寺では「石鈇山（しゃくまざん）」とも呼ぶ。'), Document(id='1e29f5ba-ae24-4fd3-a89f-1b236ea32bfc', metadata={'source': '/content/docs.json', 'seq_num': 1, 'start_index': 0}, page_content='富士山（ふじさん）は、静岡県（富士宮市、富士市、裾野市、御殿場市、駿東郡小山町）と山梨県（富士吉田市、南都留郡鳴沢村）に跨る活火山である。標高3776.12 m、日本最高峰（剣ヶ峰）の独立峰で、その優美な風貌は日本国外でも日本の象徴として広く知られている。 数多くの芸術作品の題材とされ芸術面のみならず、気候や地層など地質学的にも社会に大きな影響を与えている。懸垂曲線の山容を有した玄武岩質成層火山で構成され、その山体は駿河湾の海岸まで及ぶ。'), Document(id='de62dd66-856b-4189-bd80-0cf93579b782', metadata={'source': '/content/docs.json', 'seq_num': 96, 'start_index': 0}, page_content='四阿山（あずまやさん）は、長野県と群馬県の県境に跨る山。標高2,354 m。日本百名山の一つに数えられている。

四国地方で一番高い山は、愛媛県と高知県の県境にある石鎚山です。標高は1,982メートルで、四国地方で最も高い山です。
